In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from bs4 import BeautifulSoup
import requests
from pathlib import Path
import pandas as pd
import re
import os
import pickle
import sqlite3
import numpy as np
import datetime
#import html5lib
headers = {'User-Agent':'Mozilla/5.0'}
import time
import nba.bets as bet

#need 18 dec - april  
#need all of 20  
21 past feb 17  

In [3]:
nba = bet.nba()

In [4]:
nba.showTables

,type,name,tbl_name,rootpage,sql
0,table,rosters,rosters,4,"CREATE TABLE rosters(\n teamId TEXT,\n s..."
1,table,teams,teams,11,"CREATE TABLE teams(\n teamId TEXT,\n tea..."
2,table,shotCoverage,shotCoverage,12903,"CREATE TABLE ""shotCoverage"" (\n""TEAM_ID"" INTEG..."
3,table,players,players,2,"CREATE TABLE players(\n playerId TEXT,\n ..."
4,table,rbdStage,rbdStage,7,"CREATE TABLE ""rbdStage"" (\n""PLAYER_ID"" INTEGER..."
5,table,shootStage,shootStage,720,"CREATE TABLE ""shootStage"" (\n""PLAYER_ID"" INTEG..."
6,table,logStage,logStage,1890,"CREATE TABLE ""logStage"" (\n""SEASON_YEAR"" TEXT,..."
7,table,dribbleStage,dribbleStage,3506,"CREATE TABLE ""dribbleStage"" (\n""TEAM_ID"" INTEG..."


In [121]:
cols = [ct for ct,x in enumerate(test.columns) if (x not in test.columns[:ct-1].values) | (ct==0)]
test = pd.read_sql('''
SELECT *
FROM shotCoverage shc
inner join dribbleStage drb on shc.TEAM_ID = drb.TEAM_ID and shc.GAME_DATE = substr(drb.GAME_DATE,0,11)
''',nba.conn)
cols = [ct for ct,x in enumerate(test.columns) if (x not in test.columns[:ct-1].values) | (ct==0)]
test = test.iloc[:,cols]

In [122]:
test

,TEAM_ID,Restricted_Area_OPP_FGM,Restricted_Area_OPP_FGA,Mid_Range_OPP_FGM,Mid_Range_OPP_FGA,Left_Corner_3_OPP_FGM,Left_Corner_3_OPP_FGA,Right_Corner_3_OPP_FGM,Right_Corner_3_OPP_FGA,Above_the_Break_3_OPP_FGM,...,2_Dribbles_FG3M,2_Dribbles_FG3A,3-6_Dribbles_FG2M,3-6_Dribbles_FG2A,3-6_Dribbles_FG3M,3-6_Dribbles_FG3A,7+_Dribbles_FG2M,7+_Dribbles_FG2A,7+_Dribbles_FG3M,7+_Dribbles_FG3A
0,1610612743,23.0,36.0,2.0,10.0,0.0,2.0,1.0,2.0,9.0,...,0.0,1.0,13.0,22.0,0.0,1.0,3.0,5.0,0.0,1.0
1,1610612744,4.0,7.0,7.0,18.0,1.0,2.0,0.0,1.0,10.0,...,0.0,2.0,6.0,20.0,0.0,0.0,4.0,10.0,0.0,3.0
2,1610612747,18.0,26.0,8.0,15.0,1.0,3.0,3.0,4.0,10.0,...,0.0,0.0,5.0,11.0,3.0,4.0,3.0,6.0,0.0,0.0
3,1610612756,7.0,12.0,4.0,11.0,2.0,4.0,0.0,1.0,8.0,...,0.0,2.0,7.0,15.0,1.0,5.0,3.0,7.0,1.0,5.0
4,1610612737,17.0,25.0,4.0,9.0,3.0,9.0,0.0,4.0,8.0,...,0.0,1.0,6.0,8.0,0.0,1.0,1.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32745,1610612758,8.0,14.0,4.0,11.0,4.0,8.0,1.0,6.0,8.0,...,1.0,4.0,3.0,7.0,6.0,9.0,3.0,7.0,1.0,2.0
32746,1610612759,12.0,12.0,8.0,18.0,0.0,1.0,0.0,2.0,5.0,...,2.0,4.0,8.0,11.0,0.0,1.0,8.0,12.0,1.0,1.0
32747,1610612761,27.0,44.0,3.0,11.0,2.0,3.0,1.0,5.0,8.0,...,0.0,1.0,3.0,13.0,1.0,1.0,4.0,14.0,0.0,0.0
32748,1610612762,16.0,26.0,11.0,24.0,0.0,1.0,1.0,3.0,8.0,...,0.0,1.0,10.0,16.0,1.0,3.0,10.0,17.0,1.0,5.0


In [43]:
t = time.time()
pd.read_sql('select * from test',nba.conn)
print(time.time() - t)

0.21458697319030762


In [7]:
t = time.time()
pd.read_sql('''
SELECT * 
FROM shotCoverage shc
inner join dribbleStage drb on shc.TEAM_ID = drb.TEAM_ID and shc.GAME_DATE = substr(drb.GAME_DATE,0,11)
''',nba.conn)
print(time.time() - t)

1.3500609397888184


In [145]:
drb= pd.read_sql('''select *
                 from dribbleStage''',nba.conn)

In [146]:
drb

,TEAM_ID,GAME_ID,GAME_DATE,0_Dribbles_FG2M,0_Dribbles_FG2A,0_Dribbles_FG3M,0_Dribbles_FG3A,1_Dribble_FG2M,1_Dribble_FG2A,1_Dribble_FG3M,...,2_Dribbles_FG3M,2_Dribbles_FG3A,3-6_Dribbles_FG2M,3-6_Dribbles_FG2A,3-6_Dribbles_FG3M,3-6_Dribbles_FG3A,7+_Dribbles_FG2M,7+_Dribbles_FG2A,7+_Dribbles_FG3M,7+_Dribbles_FG3A
0,1610612737,0022301188,2024-04-14 00:00:00,18.0,21.0,15.0,29.0,7.0,11.0,1.0,...,0.0,0.0,11.0,17.0,2.0,3.0,4.0,8.0,1.0,3.0
1,1610612738,0022301186,2024-04-14 00:00:00,13.0,18.0,10.0,23.0,4.0,6.0,3.0,...,0.0,3.0,8.0,17.0,1.0,5.0,4.0,11.0,0.0,1.0
2,1610612739,0022301187,2024-04-14 00:00:00,8.0,11.0,15.0,30.0,5.0,7.0,1.0,...,1.0,4.0,9.0,19.0,1.0,2.0,1.0,7.0,1.0,3.0
3,1610612740,0022301195,2024-04-14 00:00:00,17.0,20.0,7.0,16.0,5.0,11.0,1.0,...,1.0,1.0,7.0,14.0,2.0,5.0,3.0,7.0,0.0,4.0
4,1610612741,0022301190,2024-04-14 00:00:00,10.0,11.0,11.0,24.0,3.0,7.0,0.0,...,0.0,2.0,13.0,22.0,0.0,0.0,4.0,12.0,1.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16495,1610612766,0021700003,2017-10-18 00:00:00,5.0,15.0,8.0,23.0,9.0,12.0,1.0,...,0.0,1.0,9.0,15.0,0.0,1.0,1.0,9.0,0.0,0.0
16496,1610612738,0021700001,2017-10-17 00:00:00,14.0,21.0,4.0,15.0,2.0,5.0,0.0,...,0.0,0.0,11.0,19.0,1.0,4.0,5.0,7.0,0.0,1.0
16497,1610612739,0021700001,2017-10-17 00:00:00,10.0,15.0,6.0,22.0,4.0,7.0,2.0,...,0.0,2.0,5.0,14.0,0.0,2.0,4.0,10.0,0.0,3.0
16498,1610612744,0021700002,2017-10-17 00:00:00,8.0,10.0,11.0,24.0,3.0,4.0,1.0,...,1.0,4.0,5.0,8.0,1.0,2.0,8.0,15.0,1.0,7.0


In [130]:
shot = pd.read_sql('''
select TEAM_ID, GAME_DATE 
from shotCoverage
''',nba.conn)

In [10]:
from nba_api.stats.static import teams
nba_teams = teams.get_teams()

# Select the dictionary for the Pacers, which contains their team ID
bos = [team for team in nba_teams if team['abbreviation'] == 'BOS'][0]
bos_id = bos['id']
print(f'boston_id: {bos_id}')

boston_id: 1610612738


In [11]:
db = sqlite3.connect('./nba/data/database/nba.db')

In [12]:
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.library.parameters import Season
from nba_api.stats.library.parameters import SeasonType


gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable = '00',
                                               date_from_nullable = '10/01/2017',
                                               season_type_nullable = 'Regular Season'
                                                
                                               )  
game_dates = gamefinder.get_data_frames()[0]['GAME_DATE'].unique()
games_dict = gamefinder.get_normalized_dict()
games = games_dict['LeagueGameFinderResults']
game_ids = list(set([v for l in games for k,v in l.items() if k=='GAME_ID']))
# game = games[0]
# game_id = game['GAME_ID']
# game_matchup = game['MATCHUP']

# print(f'Searching through {len(games)} game(s) for the game_id of {game_id} where {game_matchup}')

In [13]:
octdf = pd.DataFrame([pd.to_datetime(x) for x in game_dates  if pd.to_datetime(x).month==10],columns = ['day'])
octdf['year'] = [x.year for x in octdf.day]
starts = [pd.to_datetime(x) for x in octdf.groupby('year').day.min().values]

### Shooting information

In [17]:
from nba_api.stats.endpoints import LeagueDashPlayerShotLocations,CommonPlayerInfo,PlayerProfileV2,PlayerNextNGames,DraftCombineStats,LeagueDashPtStats,LeagueDashTeamShotLocations


In [27]:
missing = []
l = []
final = pd.DataFrame()
t = time.time()
games = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
print('starting at {}'.format(time.strftime('%H:%M')))
for ct,date in enumerate(game_dates):
    try:
        d = pd.to_datetime(date)
        season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])

        rbs = LeagueDashPtStats(pt_measure_type='Rebounding',player_or_team='Player',
                               date_from_nullable=date,
                               date_to_nullable = date,
                               season = season
                               ).get_data_frames()[0][['PLAYER_ID','TEAM_ID','OREB','OREB_CONTEST','OREB_CHANCES','OREB_CHANCE_DEFER','AVG_OREB_DIST',
                             'DREB','DREB_CONTEST','DREB_CHANCES','DREB_CHANCE_DEFER','AVG_DREB_DIST']]
        temp = rbs.merge(games[games.GAME_DATE==d],how='left',on=['TEAM_ID'])
        final = pd.concat([final,temp])
    except:
        missing.append(date)
    if ct % 18 == 0:
        time.sleep(np.random.choice(range(1,12),1)[0])
    elif (ct % 25 == 0) & (ct!=0):
        print('through {} days in {:.2f} with {:,} missing'.format(ct,((time.time() -t)/60),len(missing)))
        
    l.append(final.to_dict(orient='records'))
    if d in starts:
        final.to_sql('rbdStage',db,if_exists = 'append',index=False)
        print('\tsaved {} season rebs'.format(season))
        l = []
        final = pd.DataFrame()

print('finished at {}'.format(time.strftime('%H:%M')))
print('missing {:_}'.format(len(missing)))

starting at 06:57
through 25 days in 0.76 with 1 missing


/var/folders/29/0dqpwvqj4095gxyj5dkvym7r0000gn/T/ipykernel_48327/735847805.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final,temp])


through 50 days in 0.85 with 1 missing
through 75 days in 1.14 with 1 missing
through 100 days in 1.26 with 1 missing
through 125 days in 1.44 with 1 missing
through 150 days in 1.84 with 1 missing
	saved 2023-24 season rebs
through 175 days in 2.07 with 1 missing
through 200 days in 2.30 with 1 missing
through 225 days in 2.55 with 1 missing
through 250 days in 2.78 with 1 missing
through 275 days in 3.06 with 1 missing
through 300 days in 3.16 with 1 missing
	saved 2022-23 season rebs
through 325 days in 3.38 with 1 missing
through 350 days in 3.65 with 1 missing
through 375 days in 3.83 with 1 missing
through 400 days in 4.09 with 1 missing


/var/folders/29/0dqpwvqj4095gxyj5dkvym7r0000gn/T/ipykernel_48327/735847805.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final,temp])


through 425 days in 4.21 with 1 missing
through 475 days in 4.71 with 1 missing
	saved 2021-22 season rebs
through 500 days in 4.93 with 1 missing
through 525 days in 5.34 with 1 missing
through 550 days in 5.48 with 1 missing


/var/folders/29/0dqpwvqj4095gxyj5dkvym7r0000gn/T/ipykernel_48327/735847805.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final,temp])


through 575 days in 5.64 with 1 missing
through 600 days in 5.91 with 1 missing
through 625 days in 6.18 with 1 missing
through 650 days in 6.40 with 1 missing
through 675 days in 6.54 with 1 missing
through 700 days in 6.67 with 1 missing
through 725 days in 7.23 with 1 missing
through 750 days in 7.71 with 1 missing
through 775 days in 8.32 with 1 missing
	saved 2019-20 season rebs
through 800 days in 8.68 with 1 missing
through 825 days in 9.13 with 1 missing
through 850 days in 9.61 with 1 missing
through 875 days in 9.99 with 1 missing
through 925 days in 11.10 with 1 missing
	saved 2018-19 season rebs
through 950 days in 11.58 with 1 missing
through 975 days in 12.17 with 1 missing
through 1000 days in 12.53 with 1 missing
through 1025 days in 13.05 with 1 missing
through 1050 days in 13.50 with 1 missing
through 1075 days in 13.91 with 1 missing
through 1100 days in 14.30 with 1 missing
	saved 2017-18 season rebs
finished at 07:12
missing 1


### First Baskets

In [47]:
from nba_api.stats.endpoints import BoxScoreSummaryV2,LeagueDashPlayerShotLocations,BoxScoreTraditionalV2,BoxScoreAdvancedV2,PlayByPlayV2,LeagueDashOppPtShot,PlayerGameLogs

In [ ]:
box = BoxScoreSummaryV2('0022300061')

In [ ]:
df = LeagueDashPlayerShotLocations(date_from_nullable = date,
                             date_to_nullable = date,
                             season=season).get_data_frames()[0]
df = clean_shot_columns(df)
df['GAME_DATE'] = date
final = df.merge(temp,how='left',on=['GAME_DATE','TEAM_ID'])
l.append(final.to_dict(orient='records'))

#### player Shooting

In [45]:
missing = []

t = time.time()
print('starting at {}'.format(time.strftime('%H:%M')))
temp = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]

final = pd.DataFrame()
for ct,date in enumerate(game_dates):
    temp = temp[temp.GAME_DATE==date]
    d = pd.to_datetime(date)
    season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
    try:
        df = LeagueDashPlayerShotLocations(date_from_nullable = date,
                                     date_to_nullable = date,
                                     season=season).get_data_frames()[0]
        df = clean_shot_columns(df)
        df['GAME_DATE'] = date
        game = df.merge(temp,how='left',on=['GAME_DATE','TEAM_ID'])
        game = game.filter([col.replace('(Non_RA)','Non_RA') for col in game.columns if col!='TEAM_NAME'])
        final = pd.concat([game,final])
    except:
        missing.append(date)
    if ct % 18 == 0:
        time.sleep(np.random.choice(range(1,12),1)[0])
    elif (ct % 25 == 0) & (ct!=0):
        print('through {} days in {:.2f}'.format(ct,((time.time() -t)/60)))

    if d in starts:
        final.to_sql('shooterStage',db,if_exists='append',index=False)
        print('\tsaved {} season player shots'.format(season))
        final = pd.DataFrame()   
print('finished at {}'.format(time.strftime('%H:%M')))
print('missing {:_}'.format(len(missing)))

starting at 07:40
through 25 days in 0.32
through 50 days in 0.56
through 75 days in 0.84
through 100 days in 1.00
through 125 days in 1.23
through 150 days in 2.43
	saved 2023-24 season player shots
through 175 days in 2.72
through 200 days in 4.17
through 225 days in 5.83
through 250 days in 7.60
through 275 days in 9.41
through 300 days in 11.79
	saved 2022-23 season player shots
through 325 days in 14.07
through 350 days in 15.85
through 375 days in 18.24
through 400 days in 20.36
through 425 days in 22.30
through 475 days in 26.72
	saved 2021-22 season player shots
through 500 days in 28.67
through 525 days in 30.72
through 550 days in 32.88
through 575 days in 34.70
through 600 days in 36.61
through 625 days in 38.56
through 650 days in 40.27
through 675 days in 42.00
through 700 days in 43.88
through 725 days in 45.64
through 750 days in 47.53
through 775 days in 49.46
	saved 2019-20 season player shots
through 800 days in 51.22
through 825 days in 53.02
through 850 days in 54.9

### Shots by dribbles

In [ ]:
df = LeagueDashOppPtShot(date_from_nullable = date,
                                     date_to_nullable = date,
                                     season=season).get_data_frames()[0]

In [140]:

drib = ['0 Dribbles','1 Dribble','2 Dribbles','3-6 Dribbles','7+ Dribbles']
missing = []
final = pd.DataFrame()
t = time.time()
games = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
print('starting at {}'.format(time.strftime('%H:%M')))

for ct,date in enumerate(game_dates):
    d = pd.to_datetime(date)
    season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
    drb = pd.DataFrame()
    try:
        for dribbleCount in drib:
            drbShots = LeagueDashOppPtShot(date_from_nullable = date,
                                date_to_nullable = date,
                                           season=season,
                                dribble_range_nullable=dribbleCount).get_data_frames()[0]
            df = drbShots.filter([col for col in drbShots.columns if re.search('[2-3][A|M]$|ID$',col)!=None])
            df.columns = ['{}_{}'.format(dribbleCount.replace(' ','_'),col) if re.search('ID$',col)==None else col for col in df.columns]
            drb = pd.concat([drb,df])
            time.sleep(1)
        drb = drb.merge(games[games.GAME_DATE==d],how='left',on=['TEAM_ID'])
        drb = drb.groupby(['TEAM_ID','GAME_ID','GAME_DATE']).sum().reset_index()
    except:
        missing.append(date)
    final = pd.concat([final,drb])
    if ct % 18 == 0:
        time.sleep(np.random.choice(range(1,12),1)[0])
    elif (ct % 25 == 0) & (ct!=0):
        print('through {} days in {:.2f}'.format(ct,((time.time() -t)/60)))

    if d in starts:
        final.to_sql('dribbleStage',db,if_exists='append',index=False)
        print('\tsaved {} season drible shots'.format(season))
        final = pd.DataFrame()
print('finished at {}'.format(time.strftime('%H:%M')))
print('missing {:_}'.format(len(missing)))

starting at 13:40
through 25 days in 4.60


/var/folders/29/0dqpwvqj4095gxyj5dkvym7r0000gn/T/ipykernel_48327/308879817.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final,drb])


through 50 days in 8.87
through 75 days in 13.30
through 100 days in 17.38
through 125 days in 21.54
through 150 days in 25.92
	saved 2023-24 season drible shots
through 175 days in 30.35
through 200 days in 34.66
through 225 days in 39.17
through 250 days in 43.70
through 275 days in 48.22
through 300 days in 52.65
	saved 2022-23 season drible shots
through 325 days in 57.00
through 350 days in 61.23
through 375 days in 65.39
through 400 days in 69.82


/var/folders/29/0dqpwvqj4095gxyj5dkvym7r0000gn/T/ipykernel_48327/308879817.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final = pd.concat([final,drb])


through 425 days in 73.98
through 475 days in 82.63
	saved 2021-22 season drible shots
through 500 days in 86.80
through 525 days in 91.13
through 550 days in 95.26
through 575 days in 99.37
through 600 days in 103.69
through 625 days in 107.88
through 650 days in 112.16
through 675 days in 116.44
through 700 days in 120.63
through 725 days in 124.82
through 750 days in 129.07
through 775 days in 133.24
	saved 2019-20 season drible shots
through 800 days in 137.57
through 825 days in 141.85
through 850 days in 146.15
through 875 days in 150.41
through 925 days in 158.94
	saved 2018-19 season drible shots
through 950 days in 163.19
through 975 days in 167.61
through 1000 days in 171.72
through 1025 days in 175.91
through 1050 days in 180.22
through 1075 days in 184.57
through 1100 days in 189.02
	saved 2017-18 season drible shots
finished at 16:51
missing 0


In [106]:
#team defense shooting
l = []
#missing = []
t = time.time()
print('starting at {}'.format(time.strftime('%H:%M')))
final = pd.DataFrame()
for ct,date in enumerate(game_dates):
    try:
        d = pd.to_datetime(date)
        season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
        oppShots = LeagueDashTeamShotLocations(measure_type_simple='Opponent',
                                          date_from_nullable = date,
                                           date_to_nullable = date,
                                       season=season
                           ).get_data_frames()[0]
        oppShots.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in oppShots.columns]
        oppShots = oppShots.filter([col for col in oppShots.columns if re.search('_PCT$',col)==None])
        oppShots['GAME_DATE'] = date
        wide =  LeagueDashOppPtShot(date_from_nullable = date,
                          date_to_nullable = date, season=season,
                              close_def_dist_range_nullable = '6+ Feet - Wide Open').get_data_frames()[0]
        wide = wide.filter([col for col in wide.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                    (wide[col].dtype!=object)] )
        wide.columns = [col if re.search('FG',col)==None else 'WIDEOPEN_{}'.format(col) for col in wide.columns]
        wide['GAME_DATE'] = date

        op = LeagueDashOppPtShot(date_from_nullable = date,
                          date_to_nullable = date,season=season,
                              close_def_dist_range_nullable = '4-6 Feet - Open').get_data_frames()[0]
        op = op.filter([col for col in op.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                    (op[col].dtype!=object)] )
        op.columns = [col if re.search('FG',col)==None  else 'OPEN_{}'.format(col) for col in op.columns]
        op['GAME_DATE'] = date
        df = oppShots.merge(wide.merge(op,how='left',on=['TEAM_ID','GAME_DATE']),how='left',on=['TEAM_ID','GAME_DATE'])
        final = pd.concat([df,final])
        if ct % 18 == 0:
            time.sleep(np.random.choice(range(1,12),1)[0])
        elif (ct % 25 == 0) & (ct!=0):
            print('through {} days in {:.2f}'.format(ct,((time.time() -t)/60)))
        if d in starts or date==game_dates[-1]:
            final = final.filter([col.replace('(Non_RA)','Non_RA') for col in final.columns if col!='TEAM_NAME'])
            final.to_sql('shotCoverage',db,if_exists='append',index=False) 
            print('\tsaved {} season team Shots allowed'.format(season))
            final = pd.DataFrame()
    except:
        missing.append(date)
print('Seasons uploaded, missing {:,} days of games'.format(len(missing)))
#pickle.dump(l,open('nba/data/pickle/shooting2.pkl','wb'))    

starting at 20:17
through 25 days in 1.76
through 50 days in 4.26
through 75 days in 6.79
through 100 days in 9.44
through 125 days in 11.96
through 150 days in 14.44
	saved 2023-24 season team Shots allowed
through 175 days in 16.80
through 200 days in 19.18
through 225 days in 21.62
through 250 days in 23.87
through 275 days in 26.43
through 300 days in 28.88
	saved 2022-23 season team Shots allowed
through 325 days in 31.28
through 350 days in 36.06
through 375 days in 39.03
through 400 days in 41.92
through 475 days in 54.17
	saved 2021-22 season team Shots allowed
through 500 days in 56.84
through 525 days in 59.62
through 550 days in 62.33
through 575 days in 65.03
through 600 days in 67.76
through 625 days in 70.66
through 650 days in 73.52
through 675 days in 76.27
through 700 days in 79.09
through 725 days in 81.92
through 750 days in 84.66
through 775 days in 87.49
	saved 2019-20 season team Shots allowed
through 800 days in 90.52
through 825 days in 93.32
through 850 days in

,TEAM_ID,Restricted_Area_OPP_FGM,Restricted_Area_OPP_FGA,Mid_Range_OPP_FGM,Mid_Range_OPP_FGA,Left_Corner_3_OPP_FGM,Left_Corner_3_OPP_FGA,Right_Corner_3_OPP_FGM,Right_Corner_3_OPP_FGA,Above_the_Break_3_OPP_FGM,...,Corner_3_OPP_FGA,GAME_DATE,WIDEOPEN_FG2M,WIDEOPEN_FG2A,WIDEOPEN_FG3M,WIDEOPEN_FG3A,OPEN_FG2M,OPEN_FG2A,OPEN_FG3M,OPEN_FG3A
0,1610612737,20.0,33.0,6.0,13.0,2.0,4.0,1.0,5.0,7.0,...,9.0,2024-04-03,2.0,2.0,3.0,14.0,7.0,15.0,5.0,12.0
1,1610612738,18.0,25.0,3.0,6.0,0.0,3.0,1.0,2.0,4.0,...,5.0,2024-04-03,2.0,3.0,4.0,16.0,8.0,14.0,1.0,8.0
2,1610612751,20.0,31.0,6.0,7.0,1.0,2.0,4.0,7.0,5.0,...,9.0,2024-04-03,6.0,6.0,9.0,27.0,11.0,17.0,0.0,9.0
3,1610612766,18.0,40.0,8.0,16.0,1.0,5.0,0.0,1.0,6.0,...,6.0,2024-04-03,4.0,5.0,4.0,15.0,9.0,20.0,3.0,10.0
4,1610612739,9.0,17.0,4.0,9.0,2.0,3.0,2.0,5.0,14.0,...,8.0,2024-04-03,5.0,7.0,9.0,12.0,8.0,15.0,4.0,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,1610612758,15.0,24.0,2.0,6.0,1.0,4.0,1.0,5.0,5.0,...,9.0,2024-04-14,0.0,1.0,7.0,27.0,7.0,9.0,0.0,17.0
26,1610612759,15.0,24.0,5.0,12.0,3.0,7.0,1.0,3.0,4.0,...,10.0,2024-04-14,1.0,1.0,3.0,14.0,7.0,12.0,5.0,17.0
27,1610612761,26.0,35.0,3.0,6.0,1.0,4.0,2.0,6.0,6.0,...,10.0,2024-04-14,2.0,6.0,5.0,19.0,11.0,11.0,4.0,15.0
28,1610612762,18.0,25.0,4.0,9.0,5.0,8.0,1.0,2.0,8.0,...,10.0,2024-04-14,3.0,4.0,8.0,15.0,11.0,17.0,4.0,23.0


In [40]:
def clean_shot_columns(df):
    df.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in df.columns]
    df = df.filter([col for col in df.columns if (re.search('ID$|^Restricted|^Mid|^In_The|^Right|^Left|^Above',col)!=None) &
                                (re.search('PCT$',col)==None)])
    return df

In [ ]:
class data:
    #probably want to get our db connection here and the last updated date
    def __init__(self):
        conn = sqlite3.connection('nba.db')
        lastGame = conn.cursor().exucute('select max(game_date) from games')
    
    def clean_lgdash_opp_columns(df,dribble):
        df = df.filter([col for col in df.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                        (df[col].dtype!=object)] )
        if dribble != False:
            df.columns = ['{}_{}'.format(col,dribble.replace(' ','_')) if col!='TEAM_ID' else col for col in df.columns] 
        return df
    
    def clean_shot_columns(df):
        df.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in df.columns]
        df = df.filter([col for col in df.columns if (re.search('ID$|^Restricted|^Mid|^In_The|^Right|^Left|^Above',col)!=None) &
                                    (re.search('PCT$',col)==None)])
        return df
    
    
    def get_adv_box(self,gameids):
        l = []
        for gameid in gameids:
            box = BoxScoreAdvancedV3(game_id = gameid).get_data_frames()[0]
            d = box.filter([col for col in box.columns if (box[col].dtype!=object)])
            d['game_id'] = gameid
            l.append(d.to_dict(orient='records'))
        return l
    
    def get_first_buckets(self,gameid):
        for gameid in gameids:
            df = PlayByPlayV2(gameid).get_data_frames()[0]
            aind = df[(df.EVENTMSGTYPE==1) & (df.HOMEDESCRIPTION.notna())].PLAYER1_ID.idxmin()
            hind = df[(df.EVENTMSGTYPE==1) & (df.VISITORDESCRIPTION.notna())].PLAYER1_ID.idxmin()
            gd = {'gameid':gameid,'homeFirst':df.iloc[hind].PLAYER1_ID,'awayFirst':df.iloc[aind].PLAYER1_ID,
                  'first':df.iloc[min([aind,hind])].PLAYER1_ID}
        return gd
        
    def get_summary(self,gameid):
        l = []
        for gameid in gameids:
            b= BoxScoreSummaryV2(game_id=gameid).get_data_frames()
            df1 = b[5]
            df1 = df1.filter([col for col in df1.columns if (df1[col].dtype!=object) & (df1[col].sum()!=0) & (col!='GAME_SEQUENCE') ])
            df = df1.merge(b[1][['TEAM_ID','LARGEST_LEAD','PTS_FB','PTS_2ND_CHANCE','TOTAL_TURNOVERS']])
            df['GAME_ID'] = gameid
            l.append(df.to_dict(orient='records'))
        return l
        
    def get_opp_open_shot(self,dates):
        l = []
        for date in dates:
            d = pd.to_datetime(date)
            season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
            oppShots = LeagueDashTeamShotLocations(measure_type_simple='Opponent',
                                              date_from_nullable = date,
                                               date_to_nullable = date,
                                           season=season
                               ).get_data_frames()[0]
            oppShots.columns = ['{}_{}'.format(re.sub(' |-','_',a),b) if a!='' else b for a,b in oppShots.columns]
            oppShots = oppShots.filter([col for col in oppShots.columns if re.search('_PCT$',col)==None])
            oppShots['GAME_DATE'] = date
            wide =  LeagueDashOppPtShot(date_from_nullable = date,
                              date_to_nullable = date, season=season,
                                  close_def_dist_range_nullable = '6+ Feet - Wide Open').get_data_frames()[0]
            wide = wide.filter([col for col in wide.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                        (wide[col].dtype!=object)] )
            wide.columns = [col if re.search('FG',col)==None else 'WIDEOPEN_{}'.format(col) for col in wide.columns]
            wide['GAME_DATE'] = date

            op = LeagueDashOppPtShot(date_from_nullable = date,
                              date_to_nullable = date,season=season,
                                  close_def_dist_range_nullable = '4-6 Feet - Open').get_data_frames()[0]
            op = op.filter([col for col in op.columns if (re.search('_FREQUENCY$|PCT$|^G|FGM|FGA',col)==None) &
                                                        (op[col].dtype!=object)] )
            op.columns = [col if re.search('FG',col)==None  else 'OPEN_{}'.format(col) for col in op.columns]
            op['GAME_DATE'] = date
            final = oppShots.merge(wide.merge(op,how='left',on=['TEAM_ID','GAME_DATE']),how='left',on=['TEAM_ID','GAME_DATE'])
            l.append(final.to_dict(orient='records'))
        return l
        
    def get_opp_dribble_shot(self,dates):
        l = []
        drib = ['0 Dribbles','1 Dribble','2 Dribbles','3-6 Dribbles','7+ Dribbles']
        d = pd.to_datetime(date)
        season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
        drb = pd.DataFrame()
        games = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
        games.GAME_DATE = pd.to_datetime(games.GAME_DATE)
        for dribbleCount in drib:
            drbShots = LeagueDashOppPtShot(date_from_nullable = date,
                                date_to_nullable = date,
                                           season=season,
                                dribble_range_nullable=dribbleCount).get_data_frames()[0]
            df = lgDashOppColums(drbShots,dribbleCount)
            drb = pd.concat([drb,df])
            time.sleep(1)
        drb = drb.merge(games[games.GAME_DATE==d],how='left',on=['TEAM_ID'])
        drb = drb.groupby(['TEAM_ID','GAME_ID','GAME_DATE']).sum().reset_index()
        l.append(drb.to_dict(orient='records'))
        return l
    def get_player_shot_spots(self,dates):
        for date in dates:
            d = pd.to_datetime(date)
            season = '{}-{}'.format(d.year,str(d.year+1)[-2:]) if d.month>=10 else '{}-{}'.format(d.year-1,str(d.year)[-2:])
            df = LeagueDashPlayerShotLocations(date_from_nullable = date,
                                         date_to_nullable = date,
                                         season=season).get_data_frames()[0]
            df = shotcolumns(df)
            df['GAME_DATE'] = date
            temp = gamefinder.get_data_frames()[0][['GAME_DATE','TEAM_ID','GAME_ID']]
            temp = temp[temp.GAME_DATE==date]
            final = df.merge(temp,how='left',on=['GAME_DATE','TEAM_ID'])
        l.append(final.to_dict(orient='records'))
        return l
    
    def get_logs(season):
        log = PlayerGameLogs(season_nullable=season).get_data_frames()[0]
        roster = log.groupby(['SEASON_YEAR','PLAYER_ID','TEAM_ID']).GAME_DATE.agg(
                                                        firstGame = min,lastGame = max).reset_index()
        srchStr = 'SEASON_YEAR|MATCHUP|^TEAM_ABB|_PCT$|TEAM_NAME|^FG|REB$|_RANK$|WL|NICKNAME|_PTS$'
        df = log.filter([col for col in log.columns if re.search(srchStr,col)==None])
        return df
    
    def get_player_team(season):
        log = PlayerGameLogs(season_nullable=season).get_data_frames()[0]
        roster = log.groupby(['SEASON_YEAR','PLAYER_ID','TEAM_ID']).GAME_DATE.agg(
                                                        firstGame = min,lastGame = max).reset_index()
        return roster

    def cleanBaskets(df):
        test['teamFirst'] = np.where(test.homeFirst!=test['first'],test.homeFirst,test.awayFirst)
        test['gameFirst'] = np.where(test.homeFirst==test['first'],test.homeFirst,test.awayFirst)
        game = test.filter(['gameid','gameFirst'])
        game['gameFirstBasket'] = 1
        team = test.filter(['gameid','teamFirst'])
        team['teamFirstBasket'] = 1
        df = pd.concat([game,team])
        df = df.rename(columns = {'gameid':'gameId'})
        df['playerId'] = df['gameFirst'].fillna(df.teamFirst)
        df[['gameId','playerId','gameFirstBasket','teamFirstBasket']]
        return df
    

In [ ]:
#from nba_api.stats.endpoints import PlayByPlayV2 - team play-by-play
l = []
missing = []
t = time.time()
for ct,gameid in enumerate(gids):
    try:
        b= BoxScoreSummaryV2(game_id=gameid).get_data_frames()
        df1 = b[5]
        df1 = df1.filter([col for col in df1.columns if (df1[col].dtype!=object) & (df1[col].sum()!=0) & (col!='GAME_SEQUENCE') ])
        df = df1.merge(b[1][['TEAM_ID','LARGEST_LEAD','PTS_FB','PTS_2ND_CHANCE','TOTAL_TURNOVERS']])
        df['GAME_ID'] = gameid
        l.append(df.to_dict(orient='records'))
        if ct % 18 == 0:
            time.sleep(np.random.choice(range(1,12),1)[0])
        elif (ct % 100 == 0) & (ct!=0):
            print('through {} games in {:.2f}'.format(ct,((time.time() -t)/60)))
    except:
        missing.append(gameid)
print('Seasons uploaded, missing {:,} game ids'.format(len(missing)))
pickle.dump(l,open('nba/data/pickle/quarter2Info.pkl','wb'))

In [ ]:
#from nba_api.stats.endpoints import PlayByPlayV2
l = []
missing = []
t = time.time()
for ct,gameid in enumerate(game_ids):
    try:
        df = PlayByPlayV2(gameid).get_data_frames()[0]
        aind = df[(df.EVENTMSGTYPE==1) & (df.HOMEDESCRIPTION.notna())].PLAYER1_ID.idxmin()
        hind = df[(df.EVENTMSGTYPE==1) & (df.VISITORDESCRIPTION.notna())].PLAYER1_ID.idxmin()
        gd = {'gameid':gameid,'homeFirst':df.iloc[hind].PLAYER1_ID,'awayFirst':df.iloc[aind].PLAYER1_ID,
              'first':df.iloc[min([aind,hind])].PLAYER1_ID}
        l.append(gd)
        if ct % 18 == 0:
            time.sleep(np.random.choice(range(1,12),1)[0])
        elif (ct % 100 == 0) & (ct!=0):
            print('through {} games in {:.2f}'.format(ct,((time.time() -t)/60)))
        del df
    except:
        missing.append(gameid)0
print('Seasons uploaded, missing {:,} game ids'.format(len(missing)))
pickle.dump(l,open('nba/data/pickle/baskets.pkl','wb'))

## general game information

In [ ]:
from nba_api.stats.endpoints import BoxScoreSummaryV2,CommonTeamRoster,PlayerGameLogs,PlayerAwards,TeamDetails
boxSum = BoxScoreSummaryV2(game_id='0022300790')
#adv = BoxScoreAdvancedV3(game_id='0022300790')
#trad = BoxScoreTraditionalV3(game_id='0022300790')
log = PlayerGameLogs(season_nullable='2023-24')
award = PlayerAwards(player_id='203076')
tinfo = CommonTeamRoster(team_id = '1610612756')

In [ ]:
def getLogs(season):
    log = PlayerGameLogs(season_nullable=season).get_data_frames()[0]
    srchStr = 'SEASON_YEAR|MATCHUP|^TEAM_ABB|_PCT$|TEAM_NAME|^FG|REB$|_RANK$|WL|NICKNAME|_PTS$'
    df = log.filter([col for col in log.columns if re.search(srchStr,col)==None])
    return df
    

In [ ]:
pshot = pd.read_pickle('nba/data/pickle/plyrShots.pkl')
fb = pd.read_pickle('nba/data/pickle/baskets.pkl')

In [ ]:
plyShot = pd.DataFrame(columns = ['PLAYER_ID', 'TEAM_ID', 'Restricted_Area_FGM', 'Restricted_Area_FGA', 'In_The_Paint_(Non_RA)_FGM', 'In_The_Paint_(Non_RA)_FGA', 'Mid_Range_FGM', 'Mid_Range_FGA', 'Left_Corner_3_FGM', 'Left_Corner_3_FGA', 'Right_Corner_3_FGM', 'Right_Corner_3_FGA', 'Above_the_Break_3_FGM', 'Above_the_Break_3_FGA', 'GAME_DATE', 'GAME_ID'])
for l in pshot:
    for d in l:
        if d['GAME_ID']=='0022300061':
            plyShot.loc[len(plyShot)] = list(d.values())

In [ ]:
#inactive player
boxSum.get_data_frames()[3]

In [ ]:
#officials
boxSum.get_data_frames()[2]

In [ ]:
boxSum.get_data_frames()[1][['TEAM_ID','LARGEST_LEAD','PTS_FB','PTS_2ND_CHANCE','TOTAL_TURNOVERS']]

In [ ]:
b= boxSum.get_data_frames

### Prototype view
- need player log data
    - this will have the layout shown in notes
    - for now will manually filter based on team names
    - need a rolling value as well will do 5 and 10
- create a function for the fair based on their pddvalue + 5% for a vig
- kelly criterion that will take in the actual odds

In [69]:
with open('./nba/data/sql/createPlayerTable.sql','w') as file:
    f = '''
    CREATE VIEW playerGame (
    playerId TEXT,
    teamId TEXT,
    gameId TEXT,
    min FLOAT,
    resAreaM INT,
    resAreaA INT,
    paintM INT,
    paintA INT,
    midM INT,
    midA INT,
    lcThreeM INT,
    lcThreeA INT,
    rcThreeM INT,
    rcThreeA INT,
    aboveThreeM INT,
    aboveThreeA INT,
    orb INT,
    orbContest INT,
    orbChances INT,
    orbDefer INT,
    avgOrbDist FLOAT,
    drb INT,
    drbContest INT,
    drbChances INT,
    drbDefer INT,
    avgDrbDist FLOAT,
    ftm INT,
    fta INT,
    ast INT,
    tov INT,
    stl INT,
    blk INT,
    shotBlk INT,
    pf INT,
    pfd INT,
    pts INT,
    plusMinus INT,
    dd INT,
    td INT,
    usage FLOAT,
    pace FLOAT,
    poss INT,
    offRating FLOAT,
    defRating FLOAT,
    teamFirstBasket INT,
    gameFirstBasket INT,

    
    )
    
    '''
    file.write(f)

In [ ]:
with open('./nba/data/sql/createGamesTable.sql','w') as file:
    f = '''
    CREATE TABLE teamBox(
    gameId TEXT,
    gameDate TEXT,
    teamId TEXT,
    home INT,
    Q1 INT,
    Q2 INT,
    Q3 INT,
    Q4 INT,
    OT INT,
    inactive TEXT,
    pace FLOAT,
    offRating FLOAT,
    defRating FLOAT,
    PRIMARY KEY (gameId,teamId),
    FOREIGN KEY (teamId)
        REFERENCES teams(teamId)
    )
    '''
    file.write(f)

In [ ]:
with open('./nba/data/sql/createPlayerInfoTable.sql','w') as file:
    f='''
    CREATE TABLE players(
    playerId TEXT,
    playerName TEXT,
    playerHeight INT,
    playerWeight INT,
    playerBirthday TEXT,
    PRIMARY KEY (playerId)
    )
    '''
    file.write(f)

In [ ]:
with open('./nba/data/sql/createRosterTable.sql','w') as file:
    f = '''
    CREATE TABLE rosters(
    teamId TEXT,
    seasonYear TEXT,
    playerId TEXT,
    startDate TEXT,
    endDate TEXT,
    PRIMARY KEY (teamId,playerId,seasonYear),
    FOREIGN KEY (playerId) 
        REFERENCES players(playerId),
    FOREIGN KEY (teamId)
        REFERENCES teams(teamId)
    )
    
    '''
    file.write(f)

In [157]:
with open('./nba/data/sql/createTeamInfoTable.sql','w') as file:
    f = '''
    CREATE TABLE teams(
    teamId TEXT,
    teamAbrv TEXT,
    teamName TEXT,
    teamConference TEXT,
    teamDivision TEXT,
    teamLong FLOAT,
    teamLAT FLOAT,
    PRIMARY KEY (teamId)
    
    )
    '''
    file.write(f)

In [158]:
f

'\n    CREATE TABLE teams(\n    teamId TEXT,\n    teamAbrv TEXT,\n    teamName TEXT,\n    teamConference TEXT,\n    teamDivision TEXT,\n    teamLong FLOAT,\n    teamLAT FLOAT,\n    PRIMARY KEY (teamId)\n    \n    )\n    '

In [155]:
str = 'CREATE TABLE {} '
for col in ord:
    col = 'drb_'+re.sub('Dribble_|Dribbles_','',col) if col.find('Dribble')>-1 else col
    
    typ = 'TEXT' if re.search('_id|date',col.lower()) !=None else 'INT'
    str += ('{} {},'.format(col,typ)) if col != test.columns[-1] else '{} {})'.format(col,typ)
str += ' PRIMARY KEY TEAM_ID,GAME_ID'

In [156]:
str

'CREATE TABLE {} team_id TEXT,game_date TEXT,game_id TEXT,ra_fgm INT,ra_fga INT,mid_fgm INT,mid_fga INT,lc_fgm INT,lc_fga INT,rc_fgm INT,rc_fga INT,abv_fgm INT,abv_fga INT,bck_fgm INT,bck_fga INT,crn_fgm INT,crn_fga INT,wide_fg2m INT,wide_fg2a INT,wide_fg3m INT,wide_fg3a INT,open_fg2m INT,open_fg2a INT,open_fg3m INT,open_fg3a INT,drb0_fg2m INT,drb0_fg2a INT,drb0_fg3m INT,drb0_fg3a INT,drb1_fg2m INT,drb1_fg2a INT,drb1_fg3m INT,drb1_fg3a INT,drb2_fg2m INT,drb2_fg2a INT,drb2_fg3m INT,drb2_fg3a INT,drb36_fg2m INT,drb36_fg2a INT,drb36_fg3m INT,drb36_fg3a INT,drb7_fg2m INT,drb7_fg2a INT,drb7_fg3m INT,drb7_fg3m INT, PRIMARY KEY TEAM_ID,GAME_ID'

In [133]:
test.columns = ['team_id','ra_fgm','ra_fga','mid_fgm','mid_fga','lc_fgm','lc_fga','rc_fgm','rc_fga','abv_fgm','abv_fga','bck_fgm','bck_fga',
 'crn_fgm','crn_fga','game_date','wide_fg2m','wide_fg2a','wide_fg3m','wide_fg3a','open_fg2m','open_fg2a', 'open_fg3m','open_fg3a','game_id',
 'drb0_fg2m','drb0_fg2a','drb0_fg3m','drb0_fg3a','drb1_fg2m','drb1_fg2a','drb1_fg3m','drb1_fg3a','drb2_fg2m','drb2_fg2a','drb2_fg3m','drb2_fg3a',
 'drb36_fg2m', 'drb36_fg2a', 'drb36_fg3m', 'drb36_fg3a', 'drb7_fg2m','drb7_fg2a','drb7_fg3m','drb7_fg3a']



In [134]:
test.filter(['team_id','game_date','game_id','ra_fgm','ra_fga','mid_fgm','mid_fga','lc_fgm','lc_fga','rc_fgm','rc_fga','abv_fgm','abv_fga','bck_fgm','bck_fga',
 'crn_fgm','crn_fga','wide_fg2m','wide_fg2a','wide_fg3m','wide_fg3a','open_fg2m','open_fg2a', 'open_fg3m','open_fg3a',
 'drb0_fg2m','drb0_fg2a','drb0_fg3m','drb0_fg3a','drb1_fg2m','drb1_fg2a','drb1_fg3m','drb1_fg3a','drb2_fg2m','drb2_fg2a','drb2_fg3m','drb2_fg3a',
 'drb36_fg2m', 'drb36_fg2a', 'drb36_fg3m', 'drb36_fg3a', 'drb7_fg2m','drb7_fg2a','drb7_fg3m','drb7_fg3a'])

,team_id,game_date,game_id,ra_fgm,ra_fga,mid_fgm,mid_fga,lc_fgm,lc_fga,rc_fgm,...,drb2_fg3m,drb2_fg3a,drb36_fg2m,drb36_fg2a,drb36_fg3m,drb36_fg3a,drb7_fg2m,drb7_fg2a,drb7_fg3m,drb7_fg3a
0,1610612743,2023-10-24,0022300061,23.0,36.0,2.0,10.0,0.0,2.0,1.0,...,0.0,1.0,13.0,22.0,0.0,1.0,3.0,5.0,0.0,1.0
1,1610612744,2023-10-24,0022300062,4.0,7.0,7.0,18.0,1.0,2.0,0.0,...,0.0,2.0,6.0,20.0,0.0,0.0,4.0,10.0,0.0,3.0
2,1610612747,2023-10-24,0022300061,18.0,26.0,8.0,15.0,1.0,3.0,3.0,...,0.0,0.0,5.0,11.0,3.0,4.0,3.0,6.0,0.0,0.0
3,1610612756,2023-10-24,0022300062,7.0,12.0,4.0,11.0,2.0,4.0,0.0,...,0.0,2.0,7.0,15.0,1.0,5.0,3.0,7.0,1.0,5.0
4,1610612737,2023-10-25,0022300063,17.0,25.0,4.0,9.0,3.0,9.0,0.0,...,0.0,1.0,6.0,8.0,0.0,1.0,1.0,5.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32745,1610612758,2018-04-11,0021701230,8.0,14.0,4.0,11.0,4.0,8.0,1.0,...,1.0,4.0,3.0,7.0,6.0,9.0,3.0,7.0,1.0,2.0
32746,1610612759,2018-04-11,0021701226,12.0,12.0,8.0,18.0,0.0,1.0,0.0,...,2.0,4.0,8.0,11.0,0.0,1.0,8.0,12.0,1.0,1.0
32747,1610612761,2018-04-11,0021701221,27.0,44.0,3.0,11.0,2.0,3.0,1.0,...,0.0,1.0,3.0,13.0,1.0,1.0,4.0,14.0,0.0,0.0
32748,1610612762,2018-04-11,0021701229,16.0,26.0,11.0,24.0,0.0,1.0,1.0,...,0.0,1.0,10.0,16.0,1.0,3.0,10.0,17.0,1.0,5.0


In [152]:
with open('./nba/data/sql/CreateTeamShotAllowed','w') as file:
    file.write(str)

In [ ]:
'CREATE TABLE {} team_id TEXT,game_date TEXT,game_id TEXT,ra_fgm INT,ra_fga INT,mid_fgm INT,mid_fga INT,lc_fgm INT,lc_fga INT,rc_fgm INT,rc_fga INT,abv_fgm INT,abv_fga INT,bck_fgm INT,bck_fga INT,crn_fgm INT,crn_fga INT,wide_fg2m INT,wide_fg2a INT,wide_fg3m INT,wide_fg3a INT,open_fg2m INT,open_fg2a INT,open_fg3m INT,open_fg3a INT,drb0_fg2m INT,drb0_fg2a INT,drb0_fg3m INT,drb0_fg3a INT,drb1_fg2m INT,drb1_fg2a INT,drb1_fg3m INT,drb1_fg3a INT,drb2_fg2m INT,drb2_fg2a INT,drb2_fg3m INT,drb2_fg3a INT,drb36_fg2m INT,drb36_fg2a INT,drb36_fg3m INT,drb36_fg3a INT,drb7_fg2m INT,drb7_fg2a INT,drb7_fg3m INT,drb7_fg3m INT, PRIMARY KEY TEAM_ID,GAME_ID'


In [153]:
str.format('shotsAllowed')

'CREATE TABLE shotsAllowed team_id TEXT,game_date TEXT,game_id TEXT,ra_fgm INT,ra_fga INT,mid_fgm TEXT,mid_fga TEXT,lc_fgm INT,lc_fga INT,rc_fgm INT,rc_fga INT,abv_fgm INT,abv_fga INT,bck_fgm INT,bck_fga INT,crn_fgm INT,crn_fga INT,wide_fg2m TEXT,wide_fg2a TEXT,wide_fg3m TEXT,wide_fg3a TEXT,open_fg2m INT,open_fg2a INT,open_fg3m INT,open_fg3a INT,drb0_fg2m INT,drb0_fg2a INT,drb0_fg3m INT,drb0_fg3a INT,drb1_fg2m INT,drb1_fg2a INT,drb1_fg3m INT,drb1_fg3a INT,drb2_fg2m INT,drb2_fg2a INT,drb2_fg3m INT,drb2_fg3a INT,drb36_fg2m INT,drb36_fg2a INT,drb36_fg3m INT,drb36_fg3a INT,drb7_fg2m INT,drb7_fg2a INT,drb7_fg3m INT,drb7_fg3m INT, PRIMARY KEY TEAM_ID,GAME_ID'

In [154]:
nba.cur.execute(str.format('shotsAllowed'))

OperationalError: near "team_id": syntax error

In [ ]:
# cur.execute('''DROP TABLE rosters''')
# cur.execute('''DROP TABLE teams''')
cur.execute('''DROP TABLE box''')
# cur.execute('''DROP TABLE players''')
# cur.execute('''DROP TABLE games''')

In [ ]:
cur = db.cursor()

In [ ]:
# cur.execute(open('./nba/data/sql/createPlayerInfoTable.sql','r').read())
# cur.execute(open('./nba/data/sql/createRosterTable.sql','r').read())
# cur.execute(open('./nba/data/sql/createGamesTable.sql','r').read())
cur.execute(open('./nba/data/sql/createPlayerTable.sql','r').read())
# cur.execute(open('./nba/data/sql/createTeamInfoTable.sql','r').read())
db.commit()

In [ ]:
pd.read_pickle('./nba/data/pickle/')

In [ ]:
boxcols = pd.read_sql('''
PRAGMA table_info(box)
''',db).name.values

In [ ]:
shots = pd.read_pickle('./nba/data/pickle/plyrShots2023-24.pkl')

In [ ]:
PlayerGameLogs().get_data_frames()[0].columns

In [ ]:
#read in current first basket and box information clean the data and set to table'

In [ ]:
def create_box():
    '''
    Will combine advance metrics with standard box score metrics and add in first baskets
    '''
    logCols = ['SEASON_YEAR','PLAYER_ID','TEAM_ID','GAME_ID','GAME_DATE','MIN','FTM','FTA','AST','TOV','STL','BLK','BLKA','PF','PFD','PTS','PLUS_MINUS',
 'DD2','TD3']
    advcols = ['GAME_ID','PLAYER_ID','offensiveRating','defensiveRating','usagePercentage','pace','possessions']
    bskt = pd.read_pickle('./nba/data/pickle/baskets.pkl')
    bskts = set([(d['gameid'],v,1,1)  if list(d.values()).count(v) ==2 else (d['gameid'],v,1,0) for d in bskt for k,v in d.items() if type(v)!=str])
    bdf = pd.DataFrame(bskts,columns = ['GAME_ID','PLAYER_ID','teamFirst','gameFirst'])
    advbox = pd.DataFrame([r for d in pd.read_pickle('./nba/data/pickle/advbox.pkl') for r in d]).rename(columns={'game_id':'GAME_ID','personId':'PLAYER_ID'})
    advbox = advbox.filter(advcols)
    advbsk = advbox.merge(bdf,how='left',on=['GAME_ID','PLAYER_ID'])
    for season in ['2017-18','2018-19','2019-20','2020-21','2021-22','2022-23','2023-24']:
        print('{} starting at {}'.format(season,time.strftime('%H:%M')))
        seasonLog = PlayerGameLogs(season_nullable=season).get_data_frames()[0][logCols]
        seasonLog = seasonLog.filter(logCols)
        
        final = seasonLog.merge(advbsk,how='left',on = ['GAME_ID','PLAYER_ID'])
        
        final.to_sql('logs',db,if_exists='append')
    

In [ ]:
db.execute('DROP TABLE shotCoverage')
db.commit()

In [59]:
pd.read_sql("SELECT * FROM sqlite_master WHERE type='table';",db)

,type,name,tbl_name,rootpage,sql
0,table,rosters,rosters,4,"CREATE TABLE rosters(\n teamId TEXT,\n s..."
1,table,teams,teams,11,"CREATE TABLE teams(\n teamId TEXT,\n tea..."
2,table,shotCoverage,shotCoverage,12903,"CREATE TABLE ""shotCoverage"" (\n""TEAM_ID"" INTEG..."
3,table,players,players,2,"CREATE TABLE players(\n playerId TEXT,\n ..."
4,table,rbdStage,rbdStage,7,"CREATE TABLE ""rbdStage"" (\n""PLAYER_ID"" INTEGER..."
5,table,logStage,logStage,1890,"CREATE TABLE ""logStage"" (\n""index"" INTEGER,\n ..."
6,table,shootStage,shootStage,720,"CREATE TABLE ""shootStage"" (\n""PLAYER_ID"" INTEG..."


In [ ]:
df = pd.read_sql('select * from shotCoverage',db)

In [ ]:
rbs = pd.read_sql('select * from rebounds',db)

In [ ]:
rbs = rbs.drop(['index'],axis=1)

In [ ]:
db.cursor().execute('DROP TABLE games')
db.commit()

In [ ]:
def insert_data(data,table):
    cols = pd.read_sql('''
    PRAGMA table_info({})
    '''.format(table),db).name.values
    rows = data.shape[0]
    v = '?' + ',?' * (len(cols)-1)
    cur.executemany('''insert into {t} values ({v})'''.format(t=table,v=v),data.values.tolist())
    return print('{} has been updated with {:,} rows'.format(table,rows))
        

In [ ]:
insert_data(ros,'rosters')

In [ ]:
tm = nba.get_team_info()

In [ ]:
insert_data(pd.DataFrame(tm),'teams')

In [ ]:
def create_box_data(data):
    

In [ ]:
l = []
for gameid in gameids:
    df = PlayByPlayV2(gameid).get_data_frames()[0]
    aind = df[(df.EVENTMSGTYPE==1) & (df.HOMEDESCRIPTION.notna())].PLAYER1_ID.idxmin()
    hind = df[(df.EVENTMSGTYPE==1) & (df.VISITORDESCRIPTION.notna())].PLAYER1_ID.idxmin()
    gd = {'gameid':gameid,'homeFirst':1,'homePlayer':df.iloc[hind].PLAYER1_ID,'awayFirst':1,
          'awayPlayer':df.iloc[aind].PLAYER1_ID,'first':1,
          'firstPlayer':df.iloc[min([aind,hind])].PLAYER1_ID}
    l.append(gd)
df = pd.DataFrame(l)
home = df[df.homePlayer == df.firstPlayer][['gameid','homeFirst','homePlayer','first','firstPlayer']]
away = df[df.awayPlayer == df.firstPlayer][['gameid','awayFirst','awayPlayer','first','firstPlayer']]
del df

In [ ]:
h,a = nba.get_first_buckets(gameids)

In [ ]:
box = nba.get_advBox(gameids)

In [ ]:
bxdf = pd.DataFrame()

In [ ]:
bxcls = pd.read_sql('''PRAGMA table_info(box)''',db).name.values

In [ ]:
os.listdir('nba/data/pickle')

In [ ]:
bskts = pd.read_pickle('nba/data/pickle/baskets.pkl')

In [ ]:
def cleanBaskets(df):
    test['teamFirst'] = np.where(test.homeFirst!=test['first'],test.homeFirst,test.awayFirst)
    test['gameFirst'] = np.where(test.homeFirst==test['first'],test.homeFirst,test.awayFirst)
    game = test.filter(['gameid','gameFirst'])
    game['gameFirstBasket'] = 1
    team = test.filter(['gameid','teamFirst'])
    team['teamFirstBasket'] = 1
    df = pd.concat([game,team])
    df = df.rename(columns = {'gameid':'gameId'})
    df['playerId'] = df['gameFirst'].fillna(df.teamFirst)
    df[['gameId','playerId','gameFirstBasket','teamFirstBasket']]
    return df

In [ ]:
shot = pd.DataFrame([x for y in pd.read_pickle('nba/data/pickle/plyrShots.pkl')[:3] for x in y])

In [ ]:
def shot_clean():
    shot = pd.DataFrame([x for y in pd.read_pickle('nba/data/pickle/plyrShots.pkl') for x in y])
    return shot

In [ ]:
pd.DataFrame([y for x in box for y in x]).filter(['personId','game_id','usagePercentage','pace','possessions','offensiveRating','defensiveRating'])

In [ ]:
def 
def analytic_view(log,val,teams):
    '''
    Pass through the player log and the stat you are looking to evaluate to create a view to give basic information 
    on the player
    '''
    vals = [col for col in log.columns if col.find(val)>-1] + ['PLAYER_NAME','MIN']
    playing = log[log.TEAM_ABBREVIATION.isin(teams)]
    playing
    plval = playing[[val]]